In [88]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/21 14:28
# @Author  : Wang Yujia
# @File    : data_cal_np.ipynb
# @Description : 1. 计算N和P 2. 设置threshold

# 0. what for
1. 计算N和P，从`data_extract_for_asc_symmetry_2.ipynb`提取出来的
    - 计算P的方法：**method-2**，`(price-0)/bidincrement`
2. 计算完后，样本数小于`threshold`的settings不予考虑
# 1. Preparations
## 1.1 全局设置

In [89]:
# outcomes dataset
outcomes_path = "../data/outcomes_cleaned.csv"

# Output
data_np_path = r'../data/small_auctions_np.csv'                 #
settings_np_path = r'../data/small_settings_np.csv'             #
prod_path = r'../data/small_prod.csv'

settings_GT_path = "../data/small_settings_GT.csv"
settings_NN_path = "../data/small_settings_NN.csv"

# 衡量一场auction是否unique的标志
unique_setting_GT = ['bidincrement','bidfee','retail','flg_fixedprice']
unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_fixedprice']

threshold = 16

import numpy as np
import pandas as pd

## 1.2 读取data

In [90]:
outcomes = pd.read_csv(outcomes_path, encoding="utf-8")

# 2. threshold
1. threshold筛选的是某个unique_setting_NN下，拍卖发生的次数不够多，那么这个setting不具有代表性
2.

In [91]:
features = unique_setting_NN

In [92]:
# each unique setting对应了'cnt_uniq'场auction
# 注意'cnt_uniq'本来并不需要出现在最后的data中，后面用threshold会联动一下

data_grouped_tmp = outcomes.groupby(features,as_index=False)
tmp = pd.DataFrame(data_grouped_tmp.size())

# 加入大表
data_withn_cnt = pd.merge(outcomes, tmp, on=features, how="left")
data_withn_cnt.rename(columns={'size': 'cnt_uniq'}, inplace=True)
assert outcomes.shape[0] == data_withn_cnt.shape[0],"Wrong!"
data_withn_cnt.head()

,auction_id,product_id,item,desc,retail,price,bidincrement,bidfee,flg_fixedprice,id,cnt_uniq
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,0.15,0.75,0,0,69
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,0.15,0.75,0,1,60
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,0.15,0.75,0,1,60
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,0.15,0.75,0,0,69
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,0.15,0.75,0,0,69


- threshold筛选

In [93]:
# 筛选unique setting对应的样本数(data_withn_cnt['cnt_uniq'])在threshold之上的部分
data_thrld = data_withn_cnt[data_withn_cnt['cnt_uniq'] >= threshold].copy()

print("Drop 掉了 *{}* 行duplicate data".format(data_withn_cnt.shape[0] - data_thrld.shape[0]))

Drop 掉了 *12598* 行duplicate data


In [94]:
settings_NN_thrld = data_thrld[features].drop_duplicates(inplace=False)
settings_NN_thrld.reset_index(drop=True,inplace=True)
n_settings = settings_NN_thrld.shape[0]

print(f"还剩 {n_settings} 个uniq auction settings")

还剩 1226 个uniq auction settings


In [95]:
data_thrld.describe()

,auction_id,product_id,retail,price,bidincrement,bidfee,flg_fixedprice,id,cnt_uniq
count,108753.000000,1.087530e+05,108753.000000,108753.000000,108753.000000,108753.000000,108753.000000,108753.000000,108753.000000
mean,170414.271799,1.001097e+07,229.557186,43.158318,0.121971,0.699857,0.016046,595.029158,699.018519
std,44414.854122,2.055473e+03,347.499074,97.946797,0.052451,0.070762,0.125651,468.240046,1121.683898
min,86827.000000,1.000158e+07,8.590000,0.010000,0.010000,0.600000,0.000000,0.000000,16.000000
25%,132963.000000,1.001069e+07,39.990000,3.450000,0.120000,0.600000,0.000000,140.000000,61.000000
50%,166843.000000,1.001145e+07,89.990000,14.400000,0.150000,0.750000,0.000000,562.000000,224.000000
75%,208808.000000,1.001222e+07,249.990000,47.930000,0.150000,0.750000,0.000000,1027.000000,955.000000
max,259070.000000,1.001539e+07,2799.990000,5515.500000,0.240000,0.750000,1.000000,1776.000000,4998.000000


# 3. 计算N和P
## 3.1 计算N
1. 这里计算N是 **“方法二”** ：在`outcomes.tsv`中通过(price-0)/bidincrement来计算

In [96]:
data_thrld = data_thrld.copy()
data_thrld.loc[:,'N'] = data_thrld.loc[:,'price'] / (data_thrld.loc[:,'bidincrement'])
data_thrld['N'] = data_thrld['N'].astype(int)
data_thrld.head()

,auction_id,product_id,item,desc,retail,price,bidincrement,bidfee,flg_fixedprice,id,cnt_uniq,N
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,0.15,0.75,0,0,69,89
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,0.15,0.75,0,1,60,498
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,0.15,0.75,0,1,60,554
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,0.15,0.75,0,0,69,131
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,0.15,0.75,0,0,69,314


- 算完N之后，需要把fixed-price auction的bid inc变成0

In [105]:
idx_fixed = data_thrld[data_thrld.flg_fixedprice == 1].index.tolist()
data_thrld.loc[idx_fixed,'bidincrement'] = 0
data_thrld.bidincrement.unique()

array([0.15, 0.  , 0.01, 0.12, 0.24, 0.02, 0.06])

## 3.2 计算p
### 3.2.1 计算'cnt_N'
1. unique setting一样的auction认为是同一个
2. `P = cnt_n_2 / cnt_uniq`，下面需要计算`cnt_n_2`
    - `cnt_n_2`表示某个setting下的`n_2`某数值出现了几次
    - Example: `cnt_n=2`表示在某个setting下，有2场拍卖持续了n_2轮
features_cnt_n_2 = unique_setting+['n_2']

In [106]:
features = unique_setting_NN+['N']

data_grouped = data_thrld.groupby(by=features,as_index=False)
data_cnt_N = pd.DataFrame(data_grouped.size())
data_cnt_N.rename(columns={'size': 'cnt_N'}, inplace=True)

data_P = pd.merge(data_thrld,data_cnt_N,on=features,how="left")
data_P.head()
# pd.merge(data_thrld,data_cnt_N,on=features,how="")

,auction_id,product_id,item,desc,retail,price,bidincrement,bidfee,flg_fixedprice,id,cnt_uniq,N,cnt_N
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,0.15,0.75,0,0,69,89,1
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,0.15,0.75,0,1,60,498,1
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,0.15,0.75,0,1,60,554,1
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,0.15,0.75,0,0,69,131,1
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,0.15,0.75,0,0,69,314,1


### 3.2.2 计算P
- P = cnt_n / cnt_uniq
- 输出的结果在`data_withn_cnt_n12`中，其中包含了所需要的3个值：cnt_n_2 cnt_uniq

In [107]:
data_P['P'] = data_P['cnt_N'] / data_P['cnt_uniq']
data_P.head()

,auction_id,product_id,item,desc,retail,price,bidincrement,bidfee,flg_fixedprice,id,cnt_uniq,N,cnt_N,P
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,0.15,0.75,0,0,69,89,1,0.014493
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,0.15,0.75,0,1,60,498,1,0.016667
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,0.15,0.75,0,1,60,554,1,0.016667
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,0.15,0.75,0,0,69,131,1,0.014493
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,0.15,0.75,0,0,69,314,1,0.014493


- 保留部分列

In [108]:
data_P.drop(columns=['cnt_uniq','id','item'],inplace=True,axis=1)
# features_keep = unique_setting_NN +['N'] + ['P']+['cnt_N']  # 想保留的列
# data_P = data_P[features_keep].copy()
data_P.head()

,auction_id,product_id,desc,retail,price,bidincrement,bidfee,flg_fixedprice,N,cnt_N,P
0,86827,10009602,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,0.15,0.75,0,89,1,0.014493
1,87964,10009881,PSP Slim & Lite Sony Piano Black,169.99,74.70,0.15,0.75,0,498,1,0.016667
2,87965,10009881,PSP Slim & Lite Sony Piano Black,169.99,83.10,0.15,0.75,0,554,1,0.016667
3,88638,10006115,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,0.15,0.75,0,131,1,0.014493
4,88639,10006115,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,0.15,0.75,0,314,1,0.014493


# 4. 保存结果


In [109]:
data_P.to_csv(data_np_path,encoding="utf-8",index=False,header=True)
print(f"小数据集一共有 {data_P.shape[0]} 条记录")

小数据集一共有 108753 条记录


In [110]:
item_df = pd.DataFrame(data_P['desc'].drop_duplicates(inplace=False))
item_df.reset_index(drop=True,inplace=True)
item_df.to_csv(prod_path,encoding="utf-8",index=False,header=True)
item_df.head()

,desc
0,Sony Ericsson S500i Unlocked Mysterious Green
1,PSP Slim & Lite Sony Piano Black
2,iPod Touch Apple 8GB with Software Upgrade
3,LG KU990 Viewty Unlocked Black
4,Logitech Cordless Wave Keyboard and Mouse


- 保存data key

In [111]:
data_auction_GT = data_P[unique_setting_GT].copy()
settings_GT = data_auction_GT.drop_duplicates(ignore_index=True)
settings_GT.to_csv(settings_GT_path,encoding="utf-8",index=False,header=True)

print(f"当setting = {unique_setting_GT}")
print(f"一共有 *{settings_GT.shape[0]}*场不同的auction")

当setting = ['bidincrement', 'bidfee', 'retail', 'flg_fixedprice']
一共有 *627*场不同的auction


In [112]:
data_auction_NN = data_P[unique_setting_NN].copy()
settings_NN = data_auction_NN.drop_duplicates(ignore_index=True)
settings_NN.to_csv(settings_NN_path,encoding="utf-8",index=False,header=True)

print(f"当setting = {unique_setting_NN}")
print(f"一共有 *{settings_NN.shape[0]}*场不同的auction")

当setting = ['desc', 'bidincrement', 'bidfee', 'retail', 'flg_fixedprice']
一共有 *1226*场不同的auction
